In [ ]:
!pip install datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers -qq

In [ ]:
!pip install accelerate -qq

In [ ]:
!pip install fuzzywuzzy -qq

In [ ]:
!pip install python-Levenshtein -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.4 MB/s eta 0:00:00


In [ ]:
import random
from typing import Dict, List, Union

In [ ]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [ ]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\n'
        for cur in categories_:
            prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\n'
            for cur in categories_:
                prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
            prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

# Pipeline
## Загрузим модель

In [ ]:
llm_pipeline = pipeline(model='Qwen/Qwen2-1.5B-Instruct', device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

 ## Загрузим датасет

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Выделим категории

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


## Выделим случайные примеры

In [ ]:
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}\n')
    print(f'Random text: {examples_by_categories[current_category]}\n\n')

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: entertainment

Random text: В заключение турецкий танцевальный коллектив Fire of Anatolia исполнил представление "Троя".




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: geography

Random text: Темные облака, не связанные с вулканической активностью, были зафиксированы у подножия горы.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

...

Category: science/technology

Random text: Учёные работают над созданием реактора, который сможет производить энергию таким же способом.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: sports

Random text: Их отличали дисциплинированная защита, мастерство владения мячом и превосходная командная работа, и было ясно, что обыграть эту команду непросто.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: travel

Random text: Большинство районов обслуживается небольшими японскими автобусами Coaster, удобными и надежными.




## Обернем тексты в prompt для llm

In [ ]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [ ]:
print(validation_set['text'][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [ ]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: В заключение турецкий танцевальный коллектив Fire of Anatolia исполнил представление "Троя".\nВаш ответ: entertainment\nТекст: Темные облака, не связанные с вулканической активностью, были зафиксированы у подножия горы.\nВаш ответ: geography\nТекст: Его исследование показало, что если вводить гормон, то он усилит созревание легких ребёнка в период развития плода.\nВаш ответ: health\nТекст: Бахман, которая победила в проведенном в августе выборочном опросе в городе Амес, решила закончить проведение своей кампании.\nВаш ответ: politics\nТекс

## Сгенерируем ответы

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(y_pred[0])
print(y_pred[0][-1]['content'])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: В заключение турецкий танцевальный коллектив Fire of Anatolia исполнил представление "Троя".\nВаш ответ: entertainment\nТекст: Темные облака, не связанные с вулканической активностью, были зафиксированы у подножия горы.\nВаш ответ: geography\nТекст: Его исследование показало, что если вводить гормон, то он усилит созревание легких ребёнка в период развития плода.\nВаш ответ: health\nТекст: Бахман, которая победила в проведенном в августе выборочном опросе в городе Амес, решила закончить проведение своей кампании.\nВаш ответ: politics\nТекс

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                                   precision    recall  f1-score   support

                             arts       0.00      0.00      0.00         0
                    entertainment       0.80      0.44      0.57         9
                         gambling       0.00      0.00      0.00         0
                        geography       0.86      0.75      0.80         8
                           health       0.86      0.55      0.67        11
                         medicine       0.00      0.00      0.00         0
                         politics       0.82      1.00      0.90        14
               science/technology       0.82      0.92      0.87        25
                            sport       0.00      0.00      0.00         0
                           sports       1.00      0.33      0.50        12
                           travel       0.79      0.55      0.65        20
Текст: Большинство районов обслуж       0.00      0.00      0.00         0

                       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 ## Постобработка текста

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def closest_category(model, predicted_category, categories_list):
    pred_vector = model.encode(predicted_category)
    category_vectors = model.encode(categories_list)
    print(pred_vector.shape)
    similarities = cosine_similarity([pred_vector], category_vectors)[0]
    closest_index = similarities.argmax()
    return categories_list[closest_index]

In [ ]:
model.encode('trave is ok').shape

(384,)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

y_pred_with_normalization = [closest_category(model, x[-1]['content'], list_of_categories) for x in y_pred]

# y_pred_with_normalization = list(map(
#     lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
#     y_pred
# ))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)

In [ ]:
print(y_pred_with_normalization[0])

sports


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.83      0.56      0.67         9
         geography       0.86      0.75      0.80         8
            health       0.88      0.64      0.74        11
          politics       0.78      1.00      0.88        14
science/technology       0.82      0.92      0.87        25
            sports       0.67      1.00      0.80        12
            travel       0.79      0.55      0.65        20

          accuracy                           0.79        99
         macro avg       0.80      0.77      0.77        99
      weighted avg       0.80      0.79      0.78        99



## Qwen2-1.5B cos
         macro avg       0.74      0.72      0.71        99
      weighted avg       0.76      0.75      0.74        99

## Qwen2-1.5B
         macro avg       0.71      0.59      !0.61        99
      weighted avg       0.68      0.65      !0.63        99

## Qwen2-7B
         macro avg       0.77      0.78      !0.77        99
      weighted avg       0.79      0.78      !0.78        99

## Тестовые данные

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                                       precision    recall  f1-score   support

                                  art       0.00      0.00      0.00         0
                        entertainment       0.64      0.37      0.47        19
                            geography       0.88      0.82      0.85        17
                               health       0.71      0.45      0.56        22
                             medicine       0.00      0.00      0.00         0
                  music/entertainment       0.00      0.00      0.00         0
                             politics       0.84      0.90      0.87        30
                   science/technology       0.70      0.84      0.77        51
                                sport       0.00      0.00      0.00         0
                               sports       0.75      0.36      0.49        25
                               travel       0.83      0.60      0.70        40
                              weather       0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

y_pred_with_normalization = [closest_category(model, x[-1]['content'], list_of_categories) for x in y_pred]

# y_pred_with_normalization = list(map(
#     lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
#     y_pred
# ))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.69      0.47      0.56        19
         geography       0.83      0.88      0.86        17
            health       0.65      0.50      0.56        22
          politics       0.82      0.90      0.86        30
science/technology       0.70      0.84      0.77        51
            sports       0.61      0.80      0.69        25
            travel       0.83      0.60      0.70        40

          accuracy                           0.73       204
         macro avg       0.73      0.71      0.71       204
      weighted avg       0.74      0.73      0.72       204



## Qwen2-1.5B cos
         macro avg       0.79      0.69      !0.71       204
      weighted avg       0.76      0.72      !0.71       204

## Qwen2-1.5B
         macro avg       0.72      0.56      !0.57       204
      weighted avg       0.68      0.63      !0.60       204

## Qwen2-7B
         macro avg       0.84      0.85      !0.85       204
      weighted avg       0.87      0.86      !0.86       204